In [ ]:
import seaborn as sns
import numpy as np
from wordcloud import WordCloud

#đọc file
df = pd.read_csv("dataset\Amazon_Product_Review_cleaned.csv")

# Thiết lập kiểu cho biểu đồ
sns.set(style="whitegrid")

# 1. Biểu đồ Missing Values
plt.figure(figsize=(10, 5))
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]  # Chỉ lấy các cột có giá trị null
sns.barplot(x=missing_values.index, y=missing_values.values, palette="Blues_r")
plt.xticks(rotation=45)
plt.title("Missing Values in Dataset")
plt.xlabel("Columns")
plt.ylabel("Count of Missing Values")
plt.show()

# 2. Biểu đồ Countplot - Phân bố nhãn cảm xúc
plt.figure(figsize=(8, 5))
sns.countplot(x=df["sentiment"], palette="coolwarm")
plt.title("Distribution of Sentiment Labels")
plt.xlabel("Sentiment Label")
plt.ylabel("Count")
plt.show()

# 3. Biểu đồ Histogram - Phân bố độ dài câu
plt.figure(figsize=(8, 5))
sns.histplot(df["review_length"], bins=50, kde=True, color="purple")
plt.title("Distribution of Review Lengths")
plt.xlabel("Number of Words in Review")
plt.ylabel("Frequency")
plt.show()

# 4. Biểu đồ Boxplot - Mối quan hệ giữa cảm xúc & độ dài
plt.figure(figsize=(8, 5))
sns.boxplot(x=df["sentiment"], y=df["review_length"], palette="Set2")
plt.title("Relationship Between Sentiment and Review Length")
plt.xlabel("Sentiment Label")
plt.ylabel("Review Length")
plt.show()

# 5. Biểu đồ Word Cloud - Từ phổ biến
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(" ".join(df[text_column]))
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud of Most Frequent Words")
plt.show()

# 2. Biểu đồ Countplot - Phân bố nhãn cảm xúc
plt.figure(figsize=(8, 5))
sns.countplot(x=df["sentiment"], palette="coolwarm")
plt.title("Distribution of Sentiment Labels")
plt.xlabel("Sentiment Label")
plt.ylabel("Count")
plt.show()

# 3. Biểu đồ Histogram - Phân bố độ dài câu
plt.figure(figsize=(8, 5))
sns.histplot(df["review_length"], bins=50, kde=True, color="purple")
plt.title("Distribution of Review Lengths")
plt.xlabel("Number of Words in Review")
plt.ylabel("Frequency")
plt.show()

# 4. Biểu đồ Boxplot - Mối quan hệ giữa cảm xúc & độ dài
plt.figure(figsize=(8, 5))
sns.boxplot(x=df["sentiment"], y=df["review_length"], palette="Set2")
plt.title("Relationship Between Sentiment and Review Length")
plt.xlabel("Sentiment Label")
plt.ylabel("Review Length")
plt.show()

# 5. Biểu đồ Word Cloud - Từ phổ biến
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(" ".join(df[text_column]))
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud of Most Frequent Words")
plt.show()
